# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2010/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [4]:
xpath_search = '//*[@id="content_full"]/table[1]/tbody/tr/td/form/input[6]'
xpath_session = '//tr//td[1]//a[contains(@href, "ActivityDetails")]'
xpath_titles = '//tbody//tr//td[2]//a[contains(@href, "ActivityDetails")]'
xpath_sponsors = '//tbody//tr//td[3]'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [5]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [6]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles)]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor(s)']
session_type = [item.text for item in browser.find_elements_by_xpath(xpath_type) 
                if item.text != "Type" ]
session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_titles)]


In [7]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract)

True

In [8]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [9]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [10]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,205476,Intermediate R Programming,https://ww2.amstat.org/meetings/jsm/2010/onlin...,Section for Statistical Programmers and Analysts,Continuing Education Course,205476


In [20]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [11]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2010Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [12]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
15,CE_03C,Monte Carlo and Bayesian Computation with R,https://ww2.amstat.org/meetings/jsm/2010/onlin...,"Section on Physical and Engineering Sciences, ...",Continuing Education Course,205449
16,CE_02C,Methods for Designing and Analyzing Mixture Ex...,https://ww2.amstat.org/meetings/jsm/2010/onlin...,ASA,Continuing Education Course,205507
17,CE_01C,Causal Inference,https://ww2.amstat.org/meetings/jsm/2010/onlin...,ASA,Continuing Education Course,205508
28,CE_06C,Practical Bayesian Computation,https://ww2.amstat.org/meetings/jsm/2010/onlin...,Section for Statistical Programmers and Analysts,Continuing Education Course,205481
29,CE_08C,Analysis of Clinical Trials: Theory and Applic...,https://ww2.amstat.org/meetings/jsm/2010/onlin...,Biopharmaceutical Section,Continuing Education Course,205210
30,CE_01C,Causal Inference,https://ww2.amstat.org/meetings/jsm/2010/onlin...,ASA,Continuing Education Course,205965
40,CE_10C,Bayesian Ecology: Hierarchical Modeling for Ec...,https://ww2.amstat.org/meetings/jsm/2010/onlin...,Section on Bayesian Statistical Science,Continuing Education Course,205434
43,23,Data Expo 2009: A Second Look at Flight Delays,https://ww2.amstat.org/meetings/jsm/2010/onlin...,"Section on Statistical Graphics, Section on St...",Topic Contributed Papers,205726
44,37,Analysis and Modeling of Networks,https://ww2.amstat.org/meetings/jsm/2010/onlin...,"Section on Physical and Engineering Sciences, ...",Contributed Papers,205798
45,1,"Spatial Epidemiology, GIS, and Disease Mapping",https://ww2.amstat.org/meetings/jsm/2010/onlin...,"Section on Statistics in Epidemiology, CHANCE,...",Invited Papers,205133


In [18]:
test = df['Session_URL'].iloc[28]
test

'https://ww2.amstat.org/meetings/jsm/2010/onlineprogram/ActivityDetails.cfm?SessionID=205481'

In [19]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [22]:
soup.find_all('tr')[0].find('b').text.strip()

'CE_06C'

# Run Code to extract workshops and abstract_urls

In [23]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('AbstractDetails', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[4].text)
            id_no = soup.find_all('tr')[0].find('b').text.strip()
            
            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('AbstractDetails', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[0].find('b').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [24]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [25]:
len(session_url)

4437

In [26]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [27]:
len(workshop_abstract)

38

In [28]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [29]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [30]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,Visualizing Domestic Airline Traffic with SAS ...,https://ww2.amstat.org/meetings/jsm/2010/onlin...,23 *,https://ww2.amstat.org/meetings/jsm/2010/onlin...,205726


In [31]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [32]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [33]:
df_workshops.head(3)

,Abstract_Text,Session_No,Session_URL,Session_ID
0,This course will cover R from a programming pe...,205476,https://ww2.amstat.org/meetings/jsm/2010/onlin...,205476
1,This course describes the use of the statistic...,CE_03C,https://ww2.amstat.org/meetings/jsm/2010/onlin...,205449
2,Mixture experiments involve varying the compon...,CE_02C,https://ww2.amstat.org/meetings/jsm/2010/onlin...,205507


In [34]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [35]:
df_abstracts.to_excel(r'{}\2010Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2010Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [36]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

'https://ww2.amstat.org/meetings/jsm/2010/onlineprogram/AbstractDetails.cfm?abstractid=306873'

In [37]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [44]:
re.sub(r'\S+abstractid=(\d+)', r'\1', test)

'306873'

# Code to extract abstract details from abstract URLs

In [46]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('keyword', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[2].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[11].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [47]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [48]:
df_abstracts.to_excel(r'{}\2010Abstracts.xlsx'.format(dest))

In [49]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,Visualizing Domestic Airline Traffic with SAS ...,https://ww2.amstat.org/meetings/jsm/2010/onlin...,23 *,https://ww2.amstat.org/meetings/jsm/2010/onlin...,205726,306873,"Data Expo, data analysis, statistical graphics...",Topic Contributed,Millions of flights. Hundreds of airports. Doz...


In [50]:
df_details = df_workshops.append(df_abstracts)

In [51]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [52]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [53]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [54]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [55]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [56]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [57]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [58]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [59]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2010_JSM_data.xlsx'.format(new_dest))